In [1]:
import numpy as np
import pandas as pd

import requests
import time 
import random

import urllib.request
from urllib.error import HTTPError

In [2]:
auth = ["sneakermarket","sneakers","sneakerhead","sneakerscanada",
        "adidas","yeezys","nike","newbalance"]
    
rep = ["repsneakers","sneakerreps", "wengkksneakers",
        "chanzhfsneakers","michaelsneakers"]

In [3]:
sub_reds = auth + rep                                    #combine list of subreddits


image_url = []                                           
subreddit = []


imgur     = 'http://i.imgur.com/{}{}'
page_api  = 'http://imgur.com/r/{}/new/page/{}/hit.json'
album_api = 'http://imgur.com/ajaxalbums/getimages/{}/hit.json'


for sub_red in sub_reds:                                 
    s = requests.session()                               
    s.headers['user-agent'] = 'Mozilla/5.0'              

    for i in range(5):                                   
        url = page_api.format(sub_red,i)
        print(f"retrieving url from page {i+1} of {sub_red}...")

        j = s.get(url).json()
        for entry in j['data']:                          
            if entry['ext'] == '.gif' or entry['ext'] == '.mp4':
                pass                                     
            else:
                if entry['is_album']:                    
                    url = album_api.format(entry['hash'])
                    j = s.get(url).json()
                    for image in j['data']['images']:    
                        if entry['ext'] == '.gif' or entry['ext'] == '.mp4':
                            pass                         
                        else:
                            url = imgur.format(image['hash'], image['ext'])
                            image_url.append(url)        
                            subreddit.append(sub_red)    
                else:
                    url = imgur.format(entry['hash'], entry['ext'])
                    image_url.append(url)
                    subreddit.append(sub_red)

retrieving url from page 1 of sneakermarket...
retrieving url from page 2 of sneakermarket...
retrieving url from page 3 of sneakermarket...
retrieving url from page 4 of sneakermarket...
retrieving url from page 5 of sneakermarket...
retrieving url from page 1 of sneakers...
retrieving url from page 2 of sneakers...
retrieving url from page 3 of sneakers...
retrieving url from page 4 of sneakers...
retrieving url from page 5 of sneakers...
retrieving url from page 1 of sneakerhead...
retrieving url from page 2 of sneakerhead...
retrieving url from page 3 of sneakerhead...
retrieving url from page 4 of sneakerhead...
retrieving url from page 5 of sneakerhead...
retrieving url from page 1 of sneakerscanada...
retrieving url from page 2 of sneakerscanada...
retrieving url from page 3 of sneakerscanada...
retrieving url from page 4 of sneakerscanada...
retrieving url from page 5 of sneakerscanada...
retrieving url from page 1 of adidas...
retrieving url from page 2 of adidas...
retrieving

In [4]:
print(len(image_url))
print(len(subreddit))

628
628


In [5]:
%%time


header = {'User-agent':'Mozilla/5.0'}


sub_reds = auth + rep                                     
           

all_posts = []                                            
after = None                                              

for sub in sub_reds:
    url = f'https://old.reddit.com/r/{sub}.json'
    posts = []
    pages = 50
    
    print(sub)
    for i in range(pages):                                
        print(f"page {i+1} of {pages}")
        if after == None:
            current_url = url
        else:
            current_url = url + '?after=' + after         
        
        res = requests.get(current_url, headers=header)

        if res.status_code != 200:
            print('Status error', res.status_code)
            break

        current_dict = res.json()                         
        current_posts = [p['data'] for p in current_dict['data']['children']]
        posts.extend(current_posts)                       
        after = current_dict['data']['after']             

        
        sleep_duration = random.randint(2,10)
        print(f"wait {sleep_duration} seconds...")
        time.sleep(sleep_duration)
        
    all_posts.append(posts)     

sneakermarket
page 1 of 50
wait 5 seconds...
page 2 of 50
wait 3 seconds...
page 3 of 50
wait 5 seconds...
page 4 of 50
wait 9 seconds...
page 5 of 50
wait 8 seconds...
page 6 of 50
wait 5 seconds...
page 7 of 50
wait 5 seconds...
page 8 of 50
wait 7 seconds...
page 9 of 50
wait 2 seconds...
page 10 of 50
wait 6 seconds...
page 11 of 50
wait 7 seconds...
page 12 of 50
wait 10 seconds...
page 13 of 50
wait 6 seconds...
page 14 of 50
wait 4 seconds...
page 15 of 50
wait 6 seconds...
page 16 of 50
wait 10 seconds...
page 17 of 50
wait 4 seconds...
page 18 of 50
wait 5 seconds...
page 19 of 50
wait 5 seconds...
page 20 of 50
wait 9 seconds...
page 21 of 50
wait 6 seconds...
page 22 of 50
wait 3 seconds...
page 23 of 50
wait 6 seconds...
page 24 of 50
wait 9 seconds...
page 25 of 50
wait 7 seconds...
page 26 of 50
wait 9 seconds...
page 27 of 50
wait 4 seconds...
page 28 of 50
wait 5 seconds...
page 29 of 50
wait 9 seconds...
page 30 of 50
wait 9 seconds...
page 31 of 50
wait 6 seconds...
p

In [6]:
for i in range(len(all_posts)):
    for post in all_posts[i]:
            
            image_url.append(post['url'])

            
            subreddit.append(post['subreddit'])

In [7]:
print(len(image_url))
print(len(subreddit))

14315
14315


In [8]:
is_rep = ['rep' if sub in rep else 'auth' for sub in subreddit]

In [10]:

#create new col so that we can drop non images
ext = [x.split('.')[-1] for x in image_url]

#create df
df = pd.DataFrame(list(zip(image_url,subreddit,is_rep,ext)),columns=['url','subreddit','is_rep','ext'])

#drop non img
df = df[(df['ext'] == 'jpg') | (df['ext'] == 'png')]

#convert all labels to lower
df['subreddit'] = df['subreddit'].map(lambda x: x.lower())

#check no of data
df['ext'].value_counts()

jpg    4712
png     263
Name: ext, dtype: int64

In [11]:
df['subreddit'].value_counts()

yeezys             718
newbalance         713
nike               570
repsneakers        497
adidas             490
sneakers           442
sneakermarket      433
sneakerscanada     427
sneakerhead        415
wengkksneakers     103
sneakerreps         99
michaelsneakers     68
Name: subreddit, dtype: int64

In [12]:
df.drop_duplicates('url', inplace=True)

df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

In [13]:
df['subreddit'].value_counts()

newbalance         588
yeezys             469
repsneakers        460
sneakers           442
adidas             410
sneakerscanada     304
nike               299
sneakermarket      281
sneakerhead        252
sneakerreps         86
wengkksneakers       8
michaelsneakers      4
Name: subreddit, dtype: int64

In [14]:
df['name'] = df['url'].map(lambda x: x.split('/')[-1])

In [15]:
opener=urllib.request.build_opener()
opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
urllib.request.install_opener(opener)


for i in range(df.shape[0]):
    
    dl_url = df.loc[i,"url"]
    temp_label = df.loc[i,"is_rep"]
    dl_name = df.loc[i,"name"]
    try:
      urllib.request.urlretrieve(dl_url,f"D/Images_reddit/{temp_label}/{dl_name}")
      print(f"downloading {i+1} of {df.shape[0]} images...")

    except HTTPError:
      print(f"failed to download image {i+1} of {df.shape[0]}")
      pass

FileNotFoundError: ignored